In [1]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import time

import tqdm
import pandas as pd

import re

In [142]:
# 데이터프레임 생성
naver_df = pd.DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
#options.add_argument('headless') # headless 모드 설정 (창 안 띄움)
options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')

#Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome(chrome_path, options=options)

In [143]:
query = "한라산"
start_date = "20200301"
end_date = "20211001"

In [151]:
driver.get('http://www.naver.com')
time.sleep(0.5)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query)  # query_txt는 위에서 입력한 '이재용'
element.submit()

#Step 3. "블로그" 카테고리 선택
driver.find_element_by_link_text("VIEW").click( )
driver.find_element_by_link_text("블로그").click( )

time.sleep(1)

#Step 4. 오른쪽의 검색 옵션 버튼 클릭
#**자바 스크립트로 클릭되는 것은 click()이 안먹히니 send_keys 이용하기
op = driver.find_element_by_xpath("""//*[@id="snb"]/div[1]/div/div[2]/a""")
op.send_keys(Keys.ENTER)
time.sleep(1)

#Step 5. 날짜 입력
d = driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]""")
d.send_keys(Keys.ENTER)
time.sleep(1)

# 시작 날짜 입력하기
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[1]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[18]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[3]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[1]/a""").click()
time.sleep(1)

# 종료 날짜 입력하기
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[10]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[1]/a""").click()
time.sleep(1)

# Step6. 날짜 입력 후 "적용하기" 버튼 클릭
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button""").click()
time.sleep(1)

# 1차 크롤링 (url, 제목) 시작
total_page = 0
i = 0
while total_page != 200:
    print(total_page)

    a = driver.find_element_by_css_selector("#sp_blog_{} > div > div > a".format(i+1))
    i += 1
    # url 수집
    url = a.get_attribute('href')
    
    if 'naver' in url:
        # 제목 수집
        title = a.text

        naver_df.loc[total_page] = ['nan', '네이버', query, title, 'nan', url, 'nan']

        total_page += 1

    # 스크롤 내리기
    if i % 30 == 0:
        # 스크롤 높이 가져옴
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
95
95
95
95
96
97
98
99
100
101
102
103
104
105
105
106
107
108
109
109
110
111
112
112
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
184
184
185
186
187
187
188
189
190
191
192
193
194
195
196
197
198
199


In [152]:
naver_df.tail()

,날짜,네이버/티스토리,관광 명소,제목,내용,url,글쓴이
195,nan,네이버,한라산,7월 한라산 영실코스/한라산 새벽 트레킹을 추천해,nan,https://blog.naver.com/wing0617?Redirect=Log&l...,nan
196,nan,네이버,한라산,제주도 예쁜카페 추천 한라산뷰 씨힐,nan,https://blog.naver.com/steve564?Redirect=Log&l...,nan
197,nan,네이버,한라산,[제주도여행] 한라산국립공원 영실 탐방로 등산,nan,https://blog.naver.com/andna88?Redirect=Log&lo...,nan
198,nan,네이버,한라산,(한라산설경과매화공원) 한라산설경과 서귀포매화공원20選,nan,https://blog.naver.com/jeju3377?Redirect=Log&l...,nan
199,nan,네이버,한라산,[등린이기록] 네번째 한라산 세번째 영실코스,nan,https://blog.naver.com/rimi523?Redirect=Log&lo...,nan


In [153]:
# 2차 크롤링 (블로그 제목, 포스팅 날짜, 글쓴이) 시작
for i in range(len(naver_df)):
    driver.get(naver_df.loc[i]['url'])
    driver.switch_to.frame('mainFrame')
    overlays = ".se-component.se-text.se-l-default" # 내용 크롤링
    contents = driver.find_elements_by_css_selector(overlays)

    content_list = ""

    for content in contents:
        content_list = content_list + content.text

    # 글쓴이 수집
    author = driver.find_element_by_xpath("""//*[@id="nickNameArea"]""").text
    
    # date 수집
    try:
        date = driver.find_element_by_css_selector(".se_publishDate.pcol2").text
    except:
        date = driver.find_element_by_css_selector(".date.fil5.pcol2._postAddDate").text
    
    naver_df.loc[i][['날짜', '내용', '글쓴이']] = [date, content_list, author]

In [157]:
naver_df.tail()

,날짜,네이버/티스토리,관광 명소,제목,내용,url,글쓴이
195,2021. 7. 15. 12:20,네이버,한라산,7월 한라산 영실코스/한라산 새벽 트레킹을 추천해,여름 최고의 피서지\n한라산\n새벽 트레킹을 추천해\n전날 취소된 일정으로\n잘됐다...,https://blog.naver.com/wing0617?Redirect=Log&l...,여름이 좋아
196,2021. 7. 21. 8:46,네이버,한라산,제주도 예쁜카페 추천 한라산뷰 씨힐,제주여행 마지막 일정으로 잡았던\n제주공항과 가까이 위치한 카페\n한라산뷰 씨힐 !...,https://blog.naver.com/steve564?Redirect=Log&l...,프레시
197,2021. 8. 26. 10:10,네이버,한라산,[제주도여행] 한라산국립공원 영실 탐방로 등산,#제주도여행 #제주도한라산 #제주도한라산등반 #한라산영실탐방로 #영실탐방로\n7월말...,https://blog.naver.com/andna88?Redirect=Log&lo...,센치
198,2021. 2. 18. 4:39,네이버,한라산,(한라산설경과매화공원) 한라산설경과 서귀포매화공원20選,,https://blog.naver.com/jeju3377?Redirect=Log&l...,내 고향 포구
199,2021. 8. 30. 15:27,네이버,한라산,[등린이기록] 네번째 한라산 세번째 영실코스,우중산행을 결심하긴했지만\n수중산행이되었던 영실코스이야기이번 제주여행은 우중산행을 ...,https://blog.naver.com/rimi523?Redirect=Log&lo...,도리


In [180]:
import pandas as pd

save_dir = '/Users/yeonghyeon/Documents/Study/TAVE Research/데이터 분석 프로젝트팀/2. Tourism/data'

naver_final = pd.read_csv(save_dir+'naver_final.csv')
naver_final = pd.concat([naver_final, naver_df])
naver_final.reset_index(drop=True, inplace=True)
naver_final.to_csv(save_dir+'naver_final.csv', index=False)